# 02 - Data Cleaning

## Objective
Load the latest raw snapshots for each ticker, clean and standardise the structure, then save a single tidy dataset for analysis and modelling.

## Inputs
- Raw CSV snapshots in `data/raw/<version>/`
- Version label (e.g. v1)

## Outputs
- Cleaned dataset saved to `data/processed/<version>/clean_prices_<version>_<timestamp>.csv`
- Basic data quality checks (shape, missing values)

## CRISP-DM Stage
Data Preparation

In [ ]:
# Make the project root importable (so `import src...` works in notebooks)
import sys
from pathlib import Path

PROJECT_ROOT = Path("..").resolve()  # notebooks live in jupyter_notebooks/
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print("Project root added to sys.path:", PROJECT_ROOT)

In [ ]:
from pathlib import Path

import pandas as pd

from src.config import DEFAULT_TICKERS, DEFAULT_VERSION, get_paths

In [ ]:
VERSION = DEFAULT_VERSION
TICKERS = DEFAULT_TICKERS

paths = get_paths(VERSION)
RAW_DIR = paths.raw_dir
PROCESSED_DIR = paths.processed_dir
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

print("Raw dir:", RAW_DIR)
print("Processed dir:", PROCESSED_DIR)
print("Tickers:", ", ".join(TICKERS))

In [ ]:
from src.data_processing import load_and_clean_latest_raw, save_clean_prices

In [ ]:
clean_df, file_map = load_and_clean_latest_raw(
    raw_dir=RAW_DIR,
    tickers=TICKERS,
    version=VERSION,
)

for t, p in file_map.items():
    print(t, "->", p.name)

display(clean_df.groupby("Ticker").head(3))

In [ ]:
print("Rows:", len(clean_df))
print("Tickers:", clean_df["Ticker"].unique())
print("Date range:", clean_df["Date"].min(), "to", clean_df["Date"].max())

missing = (clean_df.isna().mean().sort_values(ascending=False) * 100).round(2)
print("\nMissing % (top 10):")
print(missing.head(10))

In [ ]:
archived_path, latest_path = save_clean_prices(
    clean_df=clean_df,
    processed_dir=PROCESSED_DIR,
    version=VERSION,
)

print("\nSaved:")
print("-", archived_path)
print("-", latest_path)